In [ ]:
!pip install -U adapter-transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.4/6.4 MB 15.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 29.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 105.2 MB/s eta 0:00:00


In [ ]:
!pip install torch torchvision torchaudio -f https://download.pytorch.org/whl/cu111/torch_stable.html

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://download.pytorch.org/whl/cu111/torch_stable.html


In [ ]:
from transformers import AutoConfig, AutoModelWithHeads
import torch
from transformers import RobertaTokenizer
import transformers.adapters.composition as ac
import pandas as pd
from tqdm import tqdm
from scipy.stats import pearsonr
import random

In [ ]:
val_data = pd.read_csv('/content/drive/MyDrive/NLP_final_pro/data/messages_dev_features_ready_for_WS_2022.tsv', sep='\t')

In [ ]:
tokenizer = RobertaTokenizer.from_pretrained("roberta-base")

config = AutoConfig.from_pretrained(
    "roberta-base",
    num_labels=1
)

model = AutoModelWithHeads.from_pretrained(
    "roberta-base",
    config=config,
)

/usr/local/lib/python3.10/dist-packages/transformers/adapters/models/roberta/adapter_model.py:274: FutureWarning: This class has been renamed to `RobertaAdapterModel` in v3. Please use the new class instead as this class might be removed in a future version.
  warnings.warn(


/usr/local/lib/python3.10/dist-packages/transformers/adapters/models/roberta/adapter_model.py:252: FutureWarning: This class has been renamed to `RobertaAdapterModel` in v3. Please use the new class instead as this class might be removed in a future version.
  warnings.warn(
Some weights of the model checkpoint at roberta-base were not used when initializing RobertaModelWithHeads: ['lm_head.decoder.weight', 'lm_head.layer_norm.bias', 'lm_head.dense.weight', 'lm_head.bias', 'lm_head.dense.bias', 'lm_head.layer_norm.weight']
- This IS expected if you are initializing RobertaModelWithHeads from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModelWithHeads from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification mo

In [ ]:
""" load adapters """
empathy_adapter_path = "/content/drive/MyDrive/NLP_final_pro/trained_adapters/EMP_emotion_stack"
distress_adapter_path = "/content/drive/MyDrive/NLP_final_pro/trained_adapters/DIS_emotion_stack"

empathy_adapter = model.load_adapter(empathy_adapter_path, load_as=empathy_adapter_path.split('/')[-1])
distress_adapter = model.load_adapter(distress_adapter_path, load_as=distress_adapter_path.split('/')[-1])

In [ ]:
model.set_active_adapters(ac.Parallel(empathy_adapter, distress_adapter))
model.active_head = ['EMP_emotion_stack', 'DIS_emotion_stack']

In [ ]:
""" load pretrained adapter composition """

from transformers.adapters.composition import Fuse


fusion_path = f"/content/drive/MyDrive/NLP_final_pro/trained_adapters/EpitomeFusion-distress"

# load each individual adapter
dis_er_adapter = model.load_adapter(
                fusion_path + '/distress-emotional-reactions')
dis_ex_adapter = model.load_adapter(
                fusion_path + '/distress-explorations')
dis_ip_adapter = model.load_adapter(
                fusion_path + '/distress-interpretations')

# load adapter fusion
model.load_adapter_fusion(
    fusion_path
)

# set active adapters
model.set_active_adapters(Fuse(dis_er_adapter, dis_ex_adapter, dis_ip_adapter))

# # load head
# path, distress_adapter = model.load_head(fusion_path)

model.active_head = 'EpitomeFusion-distress'

In [ ]:
""" load pretrained adapter composition: empathy """

fusion_path = f"/content/drive/MyDrive/NLP_final_pro/trained_adapters/EpitomeFusion-empathy"

# load each individual adapter
emp_er_adapter = model.load_adapter(
                fusion_path + '/empathy-emotional-reactions'
            )
emp_ex_adapter = model.load_adapter(
                fusion_path + '/empathy-explorations'
            )
emp_ip_adapter = model.load_adapter(
                fusion_path + '/empathy-interpretations'
            )

# load adapter fusion
model.load_adapter_fusion(
    fusion_path
)

# set active adapters
model.set_active_adapters(Fuse(emp_er_adapter, emp_ex_adapter, emp_ip_adapter))

# # load head
# path, empathy_adapter = model.load_head(fusion_path)

In [ ]:
model.active_head = 'EpitomeFusion-empathy'
model.active_head

'EpitomeFusion-empathy'

In [ ]:
def predict_sen(sentence):
    tokens = tokenizer.tokenize(sentence)
    input_ids = torch.tensor(tokenizer.convert_tokens_to_ids(tokens)).to('cuda')

    # move the model to the GPU
    model.to('cuda')

    # set active adapters
    model.set_active_adapters(Fuse(dis_er_adapter, dis_ex_adapter, dis_ip_adapter))
    model.active_head = 'EpitomeFusion-distress'
    outputs = model(input_ids)
    dis_score_f = outputs.logits[0][0].tolist()

    # set active adapters
    model.set_active_adapters(Fuse(emp_er_adapter, emp_ex_adapter, emp_ip_adapter))
    model.active_head = 'EpitomeFusion-empathy'
    outputs = model(input_ids)
    emp_score_f = outputs.logits[0][0].tolist()

    # stack
    model.set_active_adapters(ac.Parallel(empathy_adapter, distress_adapter))
    model.active_head = ['EMP_emotion_stack', 'DIS_emotion_stack']
    outputs = model(input_ids)

    emp_score_s = outputs[0].logits[0][0].tolist()
    dis_score_s = outputs[1].logits[0][0].tolist()

    # move the model back to the CPU
    model.to('cpu')

    return emp_score_f, dis_score_f, emp_score_s, dis_score_s

In [ ]:
def get_pred(val_data):
  pred_emp_f = []
  pred_emp_s = []
  acu_emp = []
  pred_dis_f = []
  pred_dis_s = []
  acu_dis = []
  for idx in tqdm(range(len(val_data))):
      essay = val_data['essay'].values[idx]
      emp_score_f, dis_score_f,emp_score_s,dis_score_s = predict_sen(essay)
    
      pred_emp_f.append(emp_score_f)
      pred_emp_s.append(emp_score_s)
      acu_emp.append(val_data['empathy'].values[idx])
    
      pred_dis_f.append(dis_score_f)
      pred_dis_s.append(dis_score_s)
      acu_dis.append(val_data['distress'].values[idx])
  return pred_emp_f, pred_emp_s, acu_emp, pred_dis_f, pred_dis_s, acu_dis 

In [ ]:
def get_positive_score(l_aug,org,aug):
  wrong_list = []
  for i in range(len(org)):
    if (abs(org[i]-aug[i])>0.4):
      wrong_list.append(i)
  print("Error score:",len(wrong_list)/len(org))
  random_five = random.sample(wrong_list, 5)
  for j in random_five:
    print(val_data['essay'].values[j])
    print("score:",org[j])
    print(l_aug['essay'].values[j])
    print("score:",aug[j])

In [ ]:
def get_scores(l_aug,pred_emp_f_org, pred_emp_s_org, acu_emp, pred_dis_f_org, pred_dis_s_org, acu_dis,pred_emp_f_aug, pred_emp_s_aug, pred_dis_f_aug, pred_dis_s_aug):
  
  corr_coef_f_aug, p_value = pearsonr(pred_emp_f_aug, acu_emp)
  print("Pearson Fution empathy:", corr_coef_f_aug)

  corr_coef_s_aug, p_value = pearsonr(pred_emp_s_aug, acu_emp)
  print("Pearson Stack empathy:", corr_coef_s_aug)

  corr_coef_f_a, p_value = pearsonr(pred_dis_f_aug, acu_dis)
  print("Pearson Fution distress:", corr_coef_f_a)

  corr_coef_s_a, p_value = pearsonr(pred_dis_s_aug, acu_dis)
  print("Pearson Stack distress:", corr_coef_s_a)

  print("Fution empathy:")
  get_positive_score(l_aug,pred_emp_f_org,pred_emp_f_aug)

  print("Stack empathy:")
  get_positive_score(l_aug,pred_emp_s_org,pred_emp_s_aug)

  print("Fution distress:")
  get_positive_score(l_aug,pred_dis_f_org,pred_dis_f_aug)

  print("Stack distress:")
  get_positive_score(l_aug,pred_dis_s_org,pred_dis_s_aug)

In [ ]:
pred_emp_f_org, pred_emp_s_org, acu_emp, pred_dis_f_org, pred_dis_s_org, acu_dis = get_pred(val_data)

100%|██████████| 270/270 [03:51<00:00,  1.17it/s]


In [ ]:
corr_coef, p_value = pearsonr(pred_emp_s_org, acu_emp)

print("Pearson correlation coefficient emp stack:", corr_coef)

Pearson correlation coefficient emp stack: 0.4890706649247585


In [ ]:
corr_coef, p_value = pearsonr(pred_emp_f_org, acu_emp)

print("Pearson correlation coefficient emp fusion:", corr_coef)

Pearson correlation coefficient emp fusion: 0.5129559021014046


In [ ]:
corr_coef, p_value = pearsonr(pred_dis_s_org, acu_dis)

print("Pearson correlation coefficient dis stack:", corr_coef)

Pearson correlation coefficient dis stack: 0.4537656974443214


In [ ]:
corr_coef, p_value = pearsonr(pred_dis_f_org, acu_dis)

print("Pearson correlation coefficient dis fusion:", corr_coef)

Pearson correlation coefficient dis fusion: 0.4808846695920748


In [ ]:
t_de = pd.read_csv('/content/drive/MyDrive/NLP_final_pro/t_data/test_data_de.tsv', sep='\t')
pred_emp_f_t_de, pred_emp_s_t_de, acu_emp_t_de, pred_dis_f_t_de, pred_dis_s_t_de, acu_dis_t_de = get_pred(t_de)

100%|██████████| 270/270 [04:04<00:00,  1.10it/s]


In [ ]:
get_scores(t_de,pred_emp_f_org, pred_emp_s_org, acu_emp, pred_dis_f_org, pred_dis_s_org, acu_dis,pred_emp_f_t_de, pred_emp_s_t_de, pred_dis_f_t_de, pred_dis_s_t_de)

Pearson Fution empathy: 0.17860173401510124
Pearson Stack empathy: 0.2305897285996731
Pearson Fution distress: 0.17306933104493386
Pearson Stack distress: 0.15420336988689673
Fution empathy:
score: 0.8777777777777778
An excellent (but horrible) example of just how tricky conservation work can be. Of course we should be working to protect endangered species, but you won't be successful if you don't care for the humans involved, too. I had no idea elephants were the cause of so much misery. They should be saved, but the people who live around them must be protected as well.
score: 5.771881580352783
Ein hervorragendes (aber schreckliches) Beispiel dafür, wie knifflig Naturschutzarbeit sein kann. Natürlich sollten wir uns für den Schutz gefährdeter Arten einsetzen, aber Sie werden keinen Erfolg haben, wenn Sie sich nicht auch um die beteiligten Menschen kümmern. Ich hatte keine Ahnung, dass Elefanten die Ursache für so viel Elend sind. Sie sollten gerettet werden, aber die Menschen, die um

In [ ]:
t_sw = pd.read_csv('/content/drive/MyDrive/NLP_final_pro/t_data/test_data_sw.tsv', sep='\t')
pred_emp_f_t_sw, pred_emp_s_t_sw, acu_emp_t_sw, pred_dis_f_t_sw, pred_dis_s_t_sw, acu_dis_t_sw = get_pred(t_sw)

100%|██████████| 270/270 [02:21<00:00,  1.91it/s]


In [ ]:
get_scores(t_sw,pred_emp_f_org, pred_emp_s_org, acu_emp, pred_dis_f_org, pred_dis_s_org, acu_dis,pred_emp_f_t_sw, pred_emp_s_t_sw, pred_dis_f_t_sw, pred_dis_s_t_sw)

Pearson Fution empathy: -0.08046601390868444
Pearson Stack empathy: 0.03269215447376529
Pearson Fution distress: 0.025310048179128408
Pearson Stack distress: 0.06161715890017774
Fution empathy:
score: 0.8851851851851852
Just read an article about a breakout of a certain disease in Wisconsin, it generally affects the elderly, but scientists have no idea where it is coming from. If you have elderly loved ones in Wisconsin make sure you are checking up on them. Pretty bazaar that a usually small disease like this can break out and explode among certain populations!
score: 3.887437343597412
Soma tu nakala kuhusu kuzuka kwa ugonjwa fulani huko Wisconsin, kwa ujumla huathiri wazee, lakini wanasayansi hawajui inatoka wapi. Ikiwa una wapendwa wako wazee huko Wisconsin hakikisha kuwa unawachunguza. Bazaar nzuri ambayo kwa kawaida ugonjwa mdogo kama huu unaweza kuzuka na kulipuka miongoni mwa baadhi ya watu!
score: 5.119579792022705
This is unfortunately the consequence of conflicts within this 